In [ ]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# get user list
user_list = []
for i in range(1,11):
    url = 'http://www.op.gg/ranking/ladder/page={0}'.format(i)
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text, 'html.parser')
    users = bs.findAll("td", {"class": "ranking-table__cell ranking-table__cell--summoner"})
    for line in users:
        line = str(line)
        line = line[line.find("<span>"):line.find("</span>")].replace("<span>","")
        user_list.append(line)
user_list[:5]

In [ ]:
driver_address = 'D:\chromedriver'
## 수정할것 2가지
start_number = 1 # 시작할 위치
end_number = 100

count = 1; # start number
match_number_start = 3001 # 시작할 match number , 충분히 크게 안 겹치게만

In [ ]:
def match_data_load(user_name, match_number_start):
    file_name = "user_data_{0}.csv".format(user_name)
    
    driver = webdriver.Chrome(driver_address)## 주소바꿔주기
    driver.implicitly_wait(3)
    driver.get('http://www.op.gg/summoner/userName={0}'.format(user_name))
    
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click()
    
    game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail')
    for a in game_detail_button:
        a.click()
        time.sleep(1)
    time.sleep(3)    
    html = driver.page_source
    PlayerData = BeautifulSoup(html, 'html.parser')
#     print(PlayerData.prettify())
    team_raw = PlayerData.find_all('tbody', {'class':'Content'})
    game_raw = PlayerData.find_all('thead', {'class':'Header'})
    object_raw = PlayerData.find_all('div', {'class':'Summary'})

    win_lose = []
    champion_list = []
    name_list = []
    kill_list = []
    death_list = []
    assist_list = []
    ###
    damage_list = []
    buy_ward_list = []
    build_ward_list1 = []
    build_ward_list2 = []
    CS_list = []
    CSperMin_list = []
    object_list1 = []
    object_list2 = []
    object_list3 = []

    retry = 0
    retr = 0
    for gline in game_raw:
        game_result_raw = gline.find_all('span', {'class':'GameResult'})

        game_result = []
        for gresult in game_result_raw:
            gresult = gresult.text.strip()
            if (gresult != "승리") & (gresult != "패배"):
                retry += 1
                retr += 1
                continue
            game_result.append(gresult)
        if retr == 0:
            win_lose.append(game_result)
        else:
            retr -= 1

    for obj in object_raw:
        idx = 0
        for i in obj.find_all('div',{'class':'ObjectScore'}):
            tmp = []
            if idx == 0:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list1.append(tmp)
            elif idx == 1:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list2.append(tmp)
            elif idx == 2:
                idx = 0
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list3.append(tmp)    

    for line in team_raw:
        if retry > 0:
            retry -= 1
            continue
        teamdata = []
        team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
        team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})
        team_kill_raw = line.find_all('span',{'class':'Kill'})
        team_death_raw = line.find_all('span',{'class':'Death'})
        team_assist_raw = line.find_all('span',{'class':'Assist'})
        ### 추가한것
        team_damage_raw = line.find_all('div',{'class':'ChampionDamage'})
        team_buy_ward_raw = line.find_all('div',{'class':'Buy'})
        team_build_ward_raw = line.find_all('div',{'class':'Stats'})
        team_CS_raw = line.find_all('div',{'class':'CS'})
        team_CSperMin_raw = line.find_all('div',{'class':'CSPerMinute'})

        team_champion = []
        for champ in team_champion_raw:
            champ = champ.text.strip()
            champ = champ[:champ.find('\n')]
            team_champion.append(champ)
        champion_list.append(team_champion)

        team_name = []
        for name in team_name_raw:
            name = name.text.strip()
            team_name.append(name)
        name_list.append(team_name)

        team_kill_participation = []

        team_kill = []
        for kills in team_kill_raw:
            kills = kills.text.strip()
            team_kill.append(kills)
        kill_list.append(team_kill)

        team_death = []
        for deaths in team_death_raw:
            deaths = deaths.text.strip()
            team_death.append(deaths)
        death_list.append(team_death)

        team_assist = []
        for assists in team_assist_raw:
            assists = assists.text.strip()
            team_assist.append(assists)
        assist_list.append(team_assist)

        team_damage = []
        for damage in team_damage_raw:
            damage = damage.text.strip().replace(",","")
            team_damage.append(damage)
        damage_list.append(team_damage)   

        team_buy_ward = []
        for buy_ward in team_buy_ward_raw:
            buy_ward = buy_ward.text.strip()
            team_buy_ward.append(buy_ward)
        buy_ward_list.append(team_buy_ward)  

        team_build_ward1 = []
        team_build_ward2 = []
        for build_ward in team_build_ward_raw:
            build_ward = build_ward.text.strip().replace("\t","").replace("\n","").replace("/","")
            team_build_ward1.append(build_ward[:build_ward.find(" ")])
            team_build_ward2.append(build_ward[build_ward.find(" "):].replace(" ",""))
        build_ward_list1.append(team_build_ward1)
        build_ward_list2.append(team_build_ward2)

        team_CS = []
        for CS in team_CS_raw:
            CS = CS.text.strip()
            team_CS.append(CS)
        CS_list.append(team_CS)  

        team_CSperMin = []
        for CSperMin in team_CSperMin_raw:
            CSperMin = CSperMin.text.strip()
            team_CSperMin.append(CSperMin[build_ward.find(" "):].replace(" ","").replace("당",""))
        CSperMin_list.append(team_CSperMin)  

    win_lose = win_lose[1:]

    KDAsorted = []
    matchorder = match_number_start
    matchnumber = 1
    for order in range(int((len(win_lose)+1)/2)):
        matchnumber = matchnumber-1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(CSperMin_list[matchnumber][i]))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(CSperMin_list[matchnumber][i]))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 2
        matchorder += 1

    kda_data = pd.DataFrame(KDAsorted, columns=['MatchNumber','GameResult', 'Champion', 'Nickname', 
                                                'Kill', 'Death', 'Assist', 'Damage','Buy_Pward','Build_ward','Destroy_ward',
                                                'CS', 'CSperMin','Baron','Dragon','Tower'])
    # kda_data['KDA'] = (kda_data['Kill'] + kda_data['Assist'])/ kda_data['Death']
    # kda_data.head(10)
    
    kda_data.to_csv(file_name, encoding="utf8")
    driver.quit()
    return matchorder

In [ ]:
for user_name in user_list:
    if count == end_number+1:
        break
    if count < start_number:
        count += 1
        continue
    match_number_start = match_data_load(user_name, match_number_start)
    print("complete! count :",count,", error_start : count",count+1,", start_number", match_number_start)
    count += 1